In [259]:
using JuMP
using Gurobi
using LinearAlgebra
using DataFrames, PrettyTables
using Distributions, Random, Statistics

# Problem data
M = 1e6
c = [25.0, 50.0]                     # cost vector
w = 1.0                              # target return
ϵ = 0.1                              # violation probability
ε = 0.1                              # marginal probability     
θ = 0.1                              # Wasserstein radius
x0 = 1.3                             # contextual feature realization

# mean and covariance
μ = [0.95, 0.88, 1.45]              # [X, Y1, Y2]
Σ = [ 1.0   0.20   0.30;
      0.20   0.50   0.25;
      0.30   0.25   0.80 ]           # full covariance

3×3 Matrix{Float64}:
 1.0  0.2   0.3
 0.2  0.5   0.25
 0.3  0.25  0.8

In [260]:
# sample data set
Random.seed!(1)
mv_normal = MvNormal(μ, Σ)
N = 50
data = rand(mv_normal, N)
d = Dict()
for i in 1:N 
    d[i] = abs(x0 - data[1,i])
end

In [261]:
# partition Σ and μ
ΣXX = Σ[1, 1]
ΣXY = Σ[1, 2:3]
ΣYX = Σ[2:3, 1]
ΣYY = Σ[2:3, 2:3]

# conditional mean & covariance terms
ΣY_given_X = ΣYY .- ΣYX * ΣYX' ./ ΣXX
μY_given_X = μ[2:3] .+ ΣYX * inv(ΣXX) * (x0 - μ[1])

# sample out-of-sample data
cond_dist = MvNormal(μY_given_X, ΣY_given_X)
Random.seed!(2)
n = 10000
out_of_sample_data = rand(cond_dist, n)

2×10000 Matrix{Float64}:
 -0.00680161  1.35536  0.513005   0.797689  …  0.887506  0.136493  1.46637
  0.207381    1.62912  2.37213   -0.457829     2.28919   2.0735    0.607059

In [ ]:
M = 1e3
ϵ = 0.15                              # violation probability
ε = 1.                              # marginal probability     
θ = 0.001                             # Wasserstein radius

0.001

In [358]:
## Contextual DRCCP
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "IntFeasTol", 1e-9)

@variable(model, z[1:2] ≥ 0)
@variable(model, v[1:N] ≥ 0)
@variable(model, s[1:N] ≥ 0)
@variable(model, t ≥ 0)
@variable(model, λ)
@variable(model, ζ[1:N], Bin)


# Robust chance constraint reformulation
@constraint(model, sum(z) ≥ 0.1)
@constraint(model, ϵ * ε * N * t - ε * N * λ - sum(v) ≥ θ * N * (z'z))
@constraint(model, [i in 1:N], v[i] + λ ≥ s[i])
@constraint(model, [i in 1:N], dot(z, data[2:3, i]) - w + M * ζ[i] ≥ t - s[i] - d[i] * (z'z))
@constraint(model, [i in 1:N], M * (1 - ζ[i]) ≥ t - s[i])

# Objective
@objective(model, Min, dot(c, z))

optimize!(model)
value.(z)

Set parameter IntFeasTol to value 1e-09
Set parameter IntFeasTol to value 1e-09
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 25.0.0 25A362)

CPU model: Apple M3 Max
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
IntFeasTol  1e-09

Optimize a model with 101 rows, 154 columns and 302 nonzeros
Model fingerprint: 0x0d61df93
Model has 51 quadratic constraints
Variable types: 104 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  QMatrix range    [3e-02, 2e+00]
  QLMatrix range   [4e-02, 1e+03]
  Objective range  [2e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+03]
  QRHS range       [1e+00, 1e+00]
Presolve added 50 rows and 1 columns
Presolve time: 0.00s
Presolved: 159 rows, 158 columns, 669 nonzeros
Presolved model has 2 bilinear constraint(s)
         in product terms.
         Presolve was not able to compute smaller bounds for these 

2-element Vector{Float64}:
 1.183988034841292
 0.0

In [359]:
value(λ)

-0.0

In [360]:
if termination_status(model) == MOI.OPTIMAL
    opt_value = objective_value(model)

    # Out of sample satisfaction
    ẑ = value.(z)
    n_samples = size(out_of_sample_data, 2)
    _count = 0

    for i in 1:n_samples
        if dot(out_of_sample_data[:, i], ẑ) ≥ w
            _count += 1
        end
    end

    satisfaction = _count / n_samples
else
    opt_value = nothing 
    satisfaction = nothing
end

0.5551

In [361]:
M = 1e3
ϵ = 0.15                             # violation probability   
θ = 0.001                            # Wasserstein radius

0.001

In [362]:
## Nominal DRCCP
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "IntFeasTol", 1e-9)

@variable(model, z[1:2] ≥ 0)
@variable(model, s[1:N] ≥ 0)
@variable(model, t ≥ 0)
@variable(model, ζ[1:N], Bin)


# Robust chance constraint reformulation
@constraint(model, sum(z) ≥ 0.1)
@constraint(model, ϵ * N * t - sum(s) ≥ θ * N * (z'z))
@constraint(model, [i in 1:N], dot(z, data[2:3, i]) - w + M * ζ[i] ≥ t - s[i])
@constraint(model, [i in 1:N], M * (1 - ζ[i]) ≥ t - s[i])

# Objective
@objective(model, Min, dot(c, z))

optimize!(model)
value.(z)

Set parameter IntFeasTol to value 1e-09
Set parameter IntFeasTol to value 1e-09
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 25.0.0 25A362)

CPU model: Apple M3 Max
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
IntFeasTol  1e-09

Optimize a model with 101 rows, 103 columns and 402 nonzeros
Model fingerprint: 0xce5dfa64
Model has 1 quadratic constraint
Variable types: 53 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  QMatrix range    [5e-02, 5e-02]
  QLMatrix range   [1e+00, 8e+00]
  Objective range  [2e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+03]
Presolve time: 0.00s
Presolved: 101 rows, 103 columns, 402 nonzeros
Presolved model has 1 quadratic constraint(s)
Variable types: 53 continuous, 50 integer (50 binary)

Root relaxation: objective 2.500000e+00, 51 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Curren

2-element Vector{Float64}:
 2.732916884307503
 1.3790174985676102

In [363]:
if termination_status(model) == MOI.OPTIMAL
    opt_value = objective_value(model)

    # Out of sample satisfaction
    ẑ = value.(z)
    n_samples = size(out_of_sample_data, 2)
    _count = 0

    for i in 1:n_samples
        if dot(out_of_sample_data[:, i], ẑ) ≥ w
            _count += 1
        end
    end

    satisfaction = _count / n_samples
else
    opt_value = nothing 
    satisfaction = nothing
end

0.9323